In [2]:
%load_ext rpy2.ipython
# Turning on in-IPy R capabilities

In [3]:
%%R
library(phyloseq)
library(vegan)
library(plyr)

Loading required package: permute
Loading required package: lattice
This is vegan 2.2-1


In [4]:
%%R
#Creating the physeq object from our tree and biom table with taxonomic and metadata already included, telling it \
# that the taxonomy is from greengenes database, so it will recognize the samples
physeq = import_biom("../data/otu_table_tax_no211_meta.biom", treefilename = "../data/ssu-aln.bacteria.mask.tre",
                    parseFunction = parse_taxonomy_greengenes)

In [5]:
%%R
# Normalizes the sample counts by the total - i.e., reporting what fraction of each sample each OTU makes up.
physeq = transform_sample_counts(physeq, function(x) x / sum(x))

In [6]:
%%R
physeq

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 7770 taxa and 119 samples ]
sample_data() Sample Data:       [ 119 samples by 5 sample variables ]
tax_table()   Taxonomy Table:    [ 7770 taxa by 8 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 7770 tips and 7768 internal nodes ]


In [7]:
%%R
head(sample_data(physeq))

Sample Data:        [6 samples by 5 sample variables]:
     Plants DayAmdmt Amdmt ID Day
S009      1       01     1  D   0
S204      1       21     1  D   2
S112      0       11     1  B   1
S247      0       22     2  F   2
S026      0       00     0  A   0
S023      1       00     0  C   0


In [8]:
%%R
df = as(sample_data(physeq), "data.frame")
d = distance(physeq, method = "unifrac", weighted = TRUE)

d.adonis = adonis(d ~ sample_data(physeq)$Day + sample_data(physeq)$Amdmt, df)
d.adonis


Call:
adonis(formula = d ~ sample_data(physeq)$Day + sample_data(physeq)$Amdmt,      data = df) 

Permutation: free
Number of permutations: 999

Terms added sequentially (first to last)

                           Df SumsOfSqs  MeanSqs F.Model      R2 Pr(>F)    
sample_data(physeq)$Day     2   0.14753 0.073765  17.425 0.18595  0.001 ***
sample_data(physeq)$Amdmt   2   0.16324 0.081618  19.280 0.20575  0.001 ***
Residuals                 114   0.48260 0.004233         0.60829           
Total                     118   0.79336                  1.00000           
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [9]:
%%R
df = data.frame(sample_data(physeq))
d = distance(physeq, method = "unifrac", weighted = TRUE)

d.adonis = adonis(d ~ Day + Amdmt, df)
d.adonis


Call:
adonis(formula = d ~ Day + Amdmt, data = df) 

Permutation: free
Number of permutations: 999

Terms added sequentially (first to last)

           Df SumsOfSqs  MeanSqs F.Model      R2 Pr(>F)    
Day         2   0.15358 0.076789  17.905 0.19005  0.001 ***
Amdmt       2   0.16560 0.082800  19.307 0.20493  0.001 ***
Residuals 114   0.48891 0.004289         0.60502           
Total     118   0.80809                  1.00000           
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [35]:
%%R
#physeq.D1 = subset_samples(physeq, Day == "1")
#physeq.D2 = subset_samples(physeq, Day == "2")
#physeq.PyOM = subset_samples(physeq, Amdmt == "1")
#physeq.Stover = subset_samples(physeq, Amdmt == "2")
physeq.D1.PyOM = subset_samples(physeq, Amdmt == c("1","0") & Day == "1")
physeq.D2.PyOM = subset_samples(physeq, Amdmt == c("1","0") & Day == "2")
physeq.D1.Stover = subset_samples(physeq, Amdmt == c("2","0") & Day == "1")
physeq.D2.Stover = subset_samples(physeq, Amdmt == c("2","0") & Day == "2")
physeq.D1.Both = subset_samples(physeq, Amdmt == c("2","1") & Day == "1")
physeq.D2.Both = subset_samples(physeq, Amdmt == c("2","1") & Day == "2")

In [14]:
%%R
ado.f <- function (ps){
    df = as(sample_data(ps), "data.frame")
    d = distance(ps, method = "unifrac", weighted = TRUE)
    if (length(levels(factor(sample_data(ps)$Day)))>1) factor=sample_data(ps)$Day else factor=sample_data(ps)$Amdmt
    d.adonis = adonis(d ~ factor, df)
    d.adonis[1]
    }

In [82]:
%%R
comparisons <- c(physeq.D1.PyOM, physeq.D2.PyOM, physeq.D1.Stover, physeq.D2.Stover, physeq.D1.Both, physeq.D2.Both)
names <- c("physeq.D1.PyOM", "physeq.D2.PyOM", "physeq.D1.Stover", "physeq.D2.Stover", "physeq.D1.Both", "physeq.D2.Both")

[[1]]
phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 7770 taxa and 16 samples ]
sample_data() Sample Data:       [ 16 samples by 5 sample variables ]
tax_table()   Taxonomy Table:    [ 7770 taxa by 8 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 7770 tips and 7768 internal nodes ]

[[2]]
phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 7770 taxa and 15 samples ]
sample_data() Sample Data:       [ 15 samples by 5 sample variables ]
tax_table()   Taxonomy Table:    [ 7770 taxa by 8 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 7770 tips and 7768 internal nodes ]

[[3]]
phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 7770 taxa and 13 samples ]
sample_data() Sample Data:       [ 13 samples by 5 sample variables ]
tax_table()   Taxonomy Table:    [ 7770 taxa by 8 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 7770 tips and 7768 internal nodes ]

[[4]]
phyloseq-class experiment-level object
otu_

In [30]:
%%R
fullresults = c()
for (i in comparisons) {
  if (length(levels(factor(sample_data(i)$Day)))>1) 
     name = paste("Amdmt = ",levels(factor(sample_data(i)$Amdmt)))
      else name= paste("Day = ", levels(factor(sample_data(i)$Day)))
    m = ado.f(i)
    fullresults = c(fullresults,list(name,m))
}
fullresults

[[1]]
[1] "Day =  1"

[[2]]
[[2]]$aov.tab
Permutation: free
Number of permutations: 999

Terms added sequentially (first to last)

          Df SumsOfSqs  MeanSqs F.Model      R2 Pr(>F)    
factor     2   0.19508 0.097542  32.156 0.63479  0.001 ***
Residuals 37   0.11224 0.003033         0.36521           
Total     39   0.30732                  1.00000           
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


[[3]]
[1] "Day =  2"

[[4]]
[[4]]$aov.tab
Permutation: free
Number of permutations: 999

Terms added sequentially (first to last)

          Df SumsOfSqs   MeanSqs F.Model      R2 Pr(>F)    
factor     2  0.045748 0.0228740    13.1 0.42122  0.001 ***
Residuals 36  0.062860 0.0017461         0.57878           
Total     38  0.108608                   1.00000           
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


[[5]]
[1] "Amdmt =  1"

[[6]]
[[6]]$aov.tab
Permutation: free
Number of permutations: 999

Terms added sequentially (first t

In [91]:
%%R
fullresults = c()
n=1
for (i in comparisons) {
    name = names[n]
    n = n+1
    m = ado.f(i)
    fullresults = c(fullresults,c(name,m))
}
fullresults

[[1]]
[1] "physeq.D1.PyOM"

$aov.tab
Permutation: free
Number of permutations: 999

Terms added sequentially (first to last)

          Df SumsOfSqs   MeanSqs F.Model      R2 Pr(>F)
factor     1  0.003126 0.0031262  1.3349 0.08705  0.237
Residuals 14  0.032787 0.0023419         0.91295       
Total     15  0.035913                   1.00000       

[[3]]
[1] "physeq.D2.PyOM"

$aov.tab
Permutation: free
Number of permutations: 999

Terms added sequentially (first to last)

          Df SumsOfSqs   MeanSqs F.Model      R2 Pr(>F)   
factor     1 0.0040632 0.0040632  3.1461 0.19485  0.004 **
Residuals 13 0.0167894 0.0012915         0.80515          
Total     14 0.0208526                   1.00000          
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

[[5]]
[1] "physeq.D1.Stover"

$aov.tab
Permutation: free
Number of permutations: 999

Terms added sequentially (first to last)

          Df SumsOfSqs  MeanSqs F.Model      R2 Pr(>F)    
factor     1  0.066878 0.066878 

In [96]:
%%R
citation(package = "vegan", lib.loc = NULL)


To cite package ‘vegan’ in publications use:

  Jari Oksanen, F. Guillaume Blanchet, Roeland Kindt, Pierre Legendre,
  Peter R. Minchin, R. B. O'Hara, Gavin L. Simpson, Peter Solymos, M.
  Henry H. Stevens and Helene Wagner (2015). vegan: Community Ecology
  Package. R package version 2.2-1.
  http://CRAN.R-project.org/package=vegan

A BibTeX entry for LaTeX users is

  @Manual{,
    title = {vegan: Community Ecology Package},
    author = {Jari Oksanen and F. Guillaume Blanchet and Roeland Kindt and Pierre Legendre and Peter R. Minchin and R. B. O'Hara and Gavin L. Simpson and Peter Solymos and M. Henry H. Stevens and Helene Wagner},
    year = {2015},
    note = {R package version 2.2-1},
    url = {http://CRAN.R-project.org/package=vegan},
  }

ATTENTION: This citation information has been auto-generated from the
package DESCRIPTION file and may need manual editing, see
‘help("citation")’.

